In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests 
import os

# Caminho para o diretório de destino
download_dir = r'C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros'
chromedriver=r"C:\Users\franc\OneDrive\Documentos\TCC\dados\chromedriver.exe"

# Configurações do Chrome
chrome_options = Options()
chrome_options.headless = False  # Janela visível
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

In [ ]:
paginas=[
    'https://capital.sp.gov.br/web/mobilidade/w/passageiros-transportados-2025',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/362878',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/343693',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/322860',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/306932',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/292723',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/269652',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/247850',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/228269',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/209427',
         'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/188767',     
         ]


In [3]:
# Iniciar o driver
driver = webdriver.Chrome(chromedriver,options=chrome_options)
time.sleep(2)

links_xls = {}
for i in range(len(paginas)):
    ano = 2025 - i - 5
    print(ano)
    links_xls[ano] = []
    driver.get(paginas[i])
    time.sleep(2)  # Espera o carregamento da página (ajuste conforme necessário)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Encontra todos os <a> com href terminando em .xls ou .xlsx
    for a in soup.find_all('a', href=True):
        href = a['href']
        texto_link = a.get_text(strip=True)
        
        if (href.endswith('xls') or href.endswith('xlsx')) and texto_link.lower() != 'total':
            # Caso o link seja relativo, ajusta com base na URL da página
            if href.startswith('/'):
                base_url = paginas[i].split('//')[0] + '//' + paginas[i].split('//')[1].split('/')[0]
                href = base_url + href
            links_xls[ano].append(href)

driver.close()

C:\Users\franc\AppData\Local\Temp\ipykernel_8140\2745727008.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver,options=chrome_options)


2020
2019
2018
2017
2016
2015


In [4]:
for ano in links_xls:
    print(ano,len(links_xls[ano]))

2020 366
2019 365
2018 365
2017 365
2016 366
2015 365


In [6]:
# Agora baixamos cada um dos arquivos encontrados
for ano in links_xls:
    print(ano)
    for url in links_xls[ano]:
        try:
            response = requests.get(url)
            nome_arquivo = url.split("/")[-1]
            caminho_completo = os.path.join(rf"C:\Users\franc\OneDrive\Documentos\TCC\dados\passageiros\{ano}", nome_arquivo)

            with open(caminho_completo, "wb") as f:
                f.write(response.content)

        except Exception as e:
            print(f"Erro ao baixar {ano}: {url}: {e}")

2020
2019
2018
2017
Erro ao baixar 2017: http://http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/Passag-20171108.xls: HTTPConnectionPool(host='http', port=80): Max retries exceeded with url: //www.prefeitura.sp.gov.br/cidade/secretarias/upload/Passag-20171108.xls (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000010871570A30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
2016
Erro ao baixar 2016: http://http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/acesso_a_informacao/2016/outubro/passageiros/Passag-20161021.xls: HTTPConnectionPool(host='http', port=80): Max retries exceeded with url: //www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/acesso_a_informacao/2016/outubro/passageiros/Passag-20161021.xls (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000010871570B50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'